In [28]:
import pandas as pd
import os
import re

In [29]:
folders = [f for f in os.listdir('..') if '__profile_run' in f]
folders

['268435456_table-words__512_updates-per-core__32_cores__16_concurrency__profile_run',
 '268435456_table-words__512_updates-per-core__64_cores__1_concurrency__profile_run',
 '268435456_table-words__512_updates-per-core__1_cores__2_concurrency__profile_run',
 '268435456_table-words__512_updates-per-core__1_cores__8_concurrency__profile_run',
 '268435456_table-words__512_updates-per-core__2_cores__1_concurrency__profile_run',
 '268435456_table-words__512_updates-per-core__2_cores__4_concurrency__profile_run',
 '268435456_table-words__512_updates-per-core__2_cores__16_concurrency__profile_run',
 '268435456_table-words__512_updates-per-core__4_cores__1_concurrency__profile_run',
 '268435456_table-words__512_updates-per-core__4_cores__8_concurrency__profile_run',
 '268435456_table-words__512_updates-per-core__8_cores__2_concurrency__profile_run',
 '268435456_table-words__512_updates-per-core__8_cores__16_concurrency__profile_run',
 '268435456_table-words__512_updates-per-core__16_cores__2_c

In [30]:
def vcache_name(x,y):
    return "replicant_tb_top.testbench.DUT.py[0].podrow.px[0].pod.north_vc_x[0].north_vc_row.vc_y[%d].vc_x[%d].vc.cache.vcache_prof" % (y,x)

def vcache(vcaches_data):
    # get the min and max time
    min_time = vcaches_data['time'].min()
    max_time = vcaches_data['time'].max()
    # get the min and max global ctr
    min_global_ctr = vcaches_data['global_ctr'].min()
    max_global_ctr = vcaches_data['global_ctr'].max()
    # just keep the min and max time
    vcaches_data = vcaches_data[(vcaches_data['time']==min_time)|(vcaches_data['time']==max_time)]
    summary = vcaches_data.groupby(['time','tag','global_ctr','table_words','updates_per_core','cores','concurrency']).sum().diff().dropna()
    # mark time
    summary['time_start'] = min_time
    summary['time_end'] = max_time
    summary['time_elapsed'] = max_time-min_time
    # mark cycles
    summary['global_ctr_start'] = min_global_ctr
    summary['global_ctr_end'] = max_global_ctr
    summary['global_ctr_elapsed'] = max_global_ctr-min_global_ctr
    return summary
    
def vcore(vcores_data):
    full_summary = pd.DataFrame()
    for x in range(16):
        for y in range(8):
            # filter only this vcore
            vcore_data = vcores_data[(vcores_data['x']==x)&(vcores_data['y']==y)]
            if vcore_data.empty: continue
            # get the min and max time
            min_time = vcore_data['time'].min()
            max_time = vcore_data['time'].max()
            # get the min and max global ctr
            min_global_ctr = vcore_data['global_ctr'].min()
            max_global_ctr = vcore_data['global_ctr'].max()
            # just keep the min and max time    
            vcore_data = vcore_data[(vcore_data['time']==min_time)|(vcore_data['time']==max_time)]
            summary = vcore_data.groupby(['time','tag','x','y','global_ctr','table_words','updates_per_core','cores','concurrency']).sum().diff().dropna()
            # mark time
            summary['time_start'] = min_time
            summary['time_end'] = max_time
            summary['time_elapsed']=max_time-min_time
            # mark cycles
            summary['global_ctr_start'] = min_global_ctr
            summary['global_ctr_end'] = max_global_ctr
            summary['global_ctr_elapsed'] = max_global_ctr-min_global_ctr
            # append
            full_summary = full_summary.append(summary)
    # return the full summary
    return full_summary

def folder(fldr):
    # parse the folder name to get experiment parameters
    m = re.match(r'^(\d+)_table-words__(\d+)_updates-per-core__(\d+)_cores__(\d+)_concurrency__profile_run$', fldr)
    table_words = int(m.group(1))
    updates_per_core = int(m.group(2))
    cores = int(m.group(3))
    concurrency = int(m.group(4))    
    # read data from experiment
    vcaches_data_file = '../' + fldr + '/vcache_stats.csv'
    vcores_data_file  = '../' + fldr + '/vanilla_stats.csv'
    vcaches_data = pd.read_csv(vcaches_data_file) 
    vcores_data = pd.read_csv(vcores_data_file)
    # post process vcache data    
    vcaches_data['table_words']=table_words
    vcaches_data['updates_per_core']=updates_per_core
    vcaches_data['cores']=cores
    vcaches_data['concurrency']=concurrency
    vcache_data = vcache(vcaches_data)
    # post process vcore data
    vcores_data['table_words']=table_words
    vcores_data['updates_per_core']=updates_per_core
    vcores_data['cores']=cores
    vcores_data['concurrency']=concurrency
    vcore_data = vcore(vcores_data)
    return (vcache_data, vcore_data)

folder(folders[0])

(                                                                                instr_ld  \
 time      tag        global_ctr table_words updates_per_core cores concurrency             
 407142000 3477864944 407126     268435456   512              32    16              250.0   
 
                                                                                 instr_ld_ld  \
 time      tag        global_ctr table_words updates_per_core cores concurrency                
 407142000 3477864944 407126     268435456   512              32    16                   0.0   
 
                                                                                 instr_ld_ldu  \
 time      tag        global_ctr table_words updates_per_core cores concurrency                 
 407142000 3477864944 407126     268435456   512              32    16                    0.0   
 
                                                                                 instr_ld_lw  \
 time      tag        global_ctr table_w

In [31]:
vcache_summary_data = pd.DataFrame()
vcore_summary_data = pd.DataFrame()
for fldr in folders:
    cache,core = folder(fldr)
    vcache_summary_data = vcache_summary_data.append(cache)
    vcore_summary_data = vcore_summary_data.append(core)
vcore_summary_data

,,,,,,,,,pc_r,pc_n,cycle,instr_total,instr_fadd,instr_fsub,instr_fmul,instr_fsgnj,instr_fsgnjn,instr_fsgnjx,...,stall_fcsr,stall_remote_ld_wb,stall_ifetch_wait,stall_remote_flw_wb,time_start,time_end,time_elapsed,global_ctr_start,global_ctr_end,global_ctr_elapsed
time,tag,x,y,global_ctr,table_words,updates_per_core,cores,concurrency,,,,,,,,,,,,,,,,,,,,,
386135000,3359637504,0,0,386119,268435456,512,32,16,214.0,214.0,97948.0,5619.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,288187000,386135000,97948000,288171,386119,97948
381625000,3376414736,0,1,381609,268435456,512,32,16,197.0,197.0,93155.0,5591.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,288470000,381625000,93155000,288454,381609,93155
379011000,3393191968,0,2,378995,268435456,512,32,16,197.0,197.0,89893.0,5591.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,289118000,379011000,89893000,289102,378995,89893
374057000,3409969200,0,3,374041,268435456,512,32,16,197.0,197.0,84913.0,5591.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,289144000,374057000,84913000,289128,374041,84913
389224000,3426746432,0,4,389208,268435456,512,32,16,198.0,198.0,99691.0,5591.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,289533000,389224000,99691000,289517,389208,99691
387236000,3443523664,0,5,387220,268435456,512,32,16,210.0,210.0,97216.0,5615.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,290020000,387236000,97216000,290004,387220,97216
380164000,3460300896,0,6,380148,268435456,512,32,16,198.0,198.0,89716.0,5592.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,290448000,380164000,89716000,290432,380148,89716
390805000,3477078128,0,7,390789,268435456,512,32,16,198.0,198.0,99838.0,5592.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,290967000,390805000,99838000,290951,390789,99838
388274000,3359899776,1,0,388258,268435456,512,32,16,197.0,197.0,96993.0,5591.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,291281000,388274000,96993000,291265,388258,96993


In [32]:
vcache_summary_data.to_csv('vcache_summary.txt')

In [33]:
vcore_summary_data.to_csv('vcore_summary.txt')